In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# --- 설정 (실제 환경에 맞게 초기화 필요) ---
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_DB_PATH = "./chroma_db" 
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
def initialize_chroma_db_with_collection(embedding_function, collection_name="unified_data"):
    """
    특정 컬렉션을 명시적으로 선택하여 ChromaDB 벡터 스토어를 초기화하고,
    해당 컬렉션의 문서 개수를 점검합니다.
    """
    try:
        # Chroma 인스턴스 생성 및 컬렉션 지정
        vector_db = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embedding_function,
            collection_name=collection_name
        )
        # 내부 client와 collection 정보 점검
        client = getattr(vector_db, "_client", None)
        collection = getattr(vector_db, "_collection", None)
        if client is not None:
            print(f"ChromaDB client 연결 성공: {client}")
            # 컬렉션 목록 출력
            try:
                collections = client.list_collections()
                print(f"ChromaDB 내 컬렉션 목록: {[c.name for c in collections]}")
            except Exception as e:
                print(f"컬렉션 목록 조회 실패: {e}")
        else:
            print("ChromaDB client 객체를 찾을 수 없습니다.")

        if collection is not None:
            try:
                count = collection.count()
                print(f"선택한 컬렉션({collection_name}) 문서 개수: {count}")
            except Exception as e:
                print(f"컬렉션 문서 개수 조회 실패: {e}")
        else:
            print(f"ChromaDB collection({collection_name}) 객체를 찾을 수 없습니다.")

        return vector_db
    except Exception as e:
        print(f"ChromaDB 초기화 오류: {e}")
        return None

# 연결 및 점검 (특정 컬렉션 지정)
_ = initialize_chroma_db_with_collection(embeddings, collection_name="unified_data")

C:\Users\faeqs\AppData\Local\Temp\ipykernel_10128\4058218441.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


ChromaDB client 연결 성공: <chromadb.api.client.Client object at 0x0000027764DC9EB0>
ChromaDB 내 컬렉션 목록: ['unified_data', 'langchain']
선택한 컬렉션(unified_data) 문서 개수: 608


In [6]:
def rag_pipeline(question: str, embeddings: OpenAIEmbeddings, llm: ChatOpenAI) -> str:
    """
    Vector DB (ChromaDB)에서 특정 컬렉션을 명시적으로 연결하여 RAG 파이프라인을 실행합니다.
    """
    # 1. 임베딩 함수 준비
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    # 2. ChromaDB 인스턴스 생성 (persist_directory 지정)
    try:
        vector_db = Chroma(
            persist_directory="./chroma_db",
            embedding_function=embeddings,
            collection_name="unified_data"  # 명시적으로 컬렉션 이름 지정
        )
    except Exception as e:
        return f"ERROR: ChromaDB 인스턴스 생성 오류: {e}"

    # 3. 컬렉션 연결 확인
    # (Chroma 인스턴스가 내부적으로 컬렉션을 연결하지만, 필요시 직접 접근 가능)
    collection = getattr(vector_db, "_collection", None)
    if collection is None:
        return "ERROR: 지정한 컬렉션(unified_data)에 연결할 수 없습니다."
    else:
        try:
            count = collection.count()
            print(f"연결된 컬렉션(unified_data) 문서 개수: {count}")
            print()
        except Exception as e:
            print(f"컬렉션 문서 개수 조회 실패: {e}")

    # 4. RetrievalQA 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 5}),
    )

    # 5. 질문 실행 및 결과 생성
    try:
        result = qa_chain.invoke({"query": question})
        return result["result"]
    except Exception as e:
        return f"ERROR: RAG 파이프라인 실행 오류: {e}"

In [7]:
# RAG 파이프라인 테스트용 질문 목록
questions = [
    "Bella Roma 레스토랑의 영업시간을 알려주세요.",
    "까르보나라 파스타의 가격은 얼마인가요?",
    "주방장 이름이 뭐예요?",
    "티라미수를 구매한 이력이 있는 사용자들이 남긴 리뷰를 모두 알려줘",
    "2024년 8월에 가장 많이 팔린 메뉴는 무엇인가요?",
    "글루텐 알레르기가 있는 고객이 먹을 수 없는 메뉴는 무엇인가요?",
    "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?",
    "‘가격’이라는 키워드가 포함된 리뷰들의 평균 평점은 얼마이며, 이 리뷰를 남긴 사용자들은 주로 어떤 메뉴를 주문했나요?"
]

# 질문들을 rag_pipeline에 넣고 결과 출력
for idx, q in enumerate(questions, 1):
    print(f"\n{'='*30}\nQ{idx}. {q}\n{'-'*30}")
    answer = rag_pipeline(q, embeddings, llm)
    print(f"A{idx}. {answer}\n")


Q1. Bella Roma 레스토랑의 영업시간을 알려주세요.
------------------------------
연결된 컬렉션(unified_data) 문서 개수: 608

A1. Bella Roma의 영업시간은 다음과 같습니다.

- 평일(월~금): 11:00 ~ 22:00  
- 주말(토·일): 11:00 ~ 23:00


Q2. 까르보나라 파스타의 가격은 얼마인가요?
------------------------------
연결된 컬렉션(unified_data) 문서 개수: 608

A2. 까르보나라 파스타는 한 개에 18,000원입니다.


Q3. 주방장 이름이 뭐예요?
------------------------------
연결된 컬렉션(unified_data) 문서 개수: 608

A3. 주방장(Head Chef)은 김철수입니다. 15년 경력의 베테랑 셰프예요.


Q4. 티라미수를 구매한 이력이 있는 사용자들이 남긴 리뷰를 모두 알려줘
------------------------------
연결된 컬렉션(unified_data) 문서 개수: 608

A4. 죄송하지만 제공된 자료에는 리뷰 내용이 포함되어 있지 않습니다. 따라서 티라미수를 구매한 사용자들이 남긴 리뷰를 확인할 수 없습니다.

참고로 제공된 구매 기록에 따르면 티라미수를 구매한 사용자는 다음과 같습니다.
- 사용자4 (2024-02-11, 3개)  
- 사용자4 (2024-10-07, 3개)  
- 사용자47 (2024-05-23, 3개)  
- 사용자3 (2024-01-29, 2개)  
- 사용자41 (2024-04-05, 3개)  

원하시면 다음 중 하나를 도와드리겠습니다.
- 리뷰 데이터(파일 또는 데이터베이스 접근 정보)를 제공해 주시면 리뷰를 모두 추출해 드리겠습니다.  
- 리뷰가 저장된 위치를 알려주시면 조회 방법을 안내해 드리겠습니다.  
- 위 구매자들에 대한 추가 요약(구매 횟수/총액 등)을 제공해 드리겠습니다.


Q5. 2024년 8월에 가장 많이 팔린 메뉴